In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
matplotlib.use('TkAgg')# change backend to show
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
def compute_acc(vxs,vys):
    global prediction
    ypre=sess.run(prediction,feed_dict={x:vxs})
    corrpre=tf.equal(tf.arg_max(ypre,1),tf.arg_max(vys,1))
    acc=tf.reduce_mean(tf.cast(x=corrpre,dtype=tf.float32))
    re=sess.run(acc,feed_dict={x:vxs,y:vys})
    return re
def compute_loss(vxs,vys):
    global prediction
    ypre=sess.run(loss,feed_dict={x:vxs})
    #corrpre=tf.equal(tf.arg_max(ypre,1),tf.arg_max(vys,1))
    acc=tf.reduce_mean(ypre)
    re=sess.run(acc,feed_dict={x:vxs,y:vys})
    return re

In [3]:
#paras
lr=0.001
train_iters=10000
batch_size=126
dis_step=50

n_inputs=28
n_steps=28
n_hidden_units=128
n_classes=10


In [4]:
x=tf.placeholder(dtype=tf.float32,shape=[None,n_steps,n_inputs])
y=tf.placeholder(dtype=tf.float32,shape=[None,n_classes])
init_state = tf.placeholder(tf.float32, [None, 2*n_hidden_units])
weights={
    'in':tf.Variable(tf.random_normal(shape=[n_inputs,n_hidden_units])),
    'out':tf.Variable(tf.random_normal(shape=[n_hidden_units,n_classes]))
}
biases={
    'in':tf.Variable(tf.random_normal(shape=[n_hidden_units])),
    'out':tf.Variable(tf.random_normal(shape=[n_classes]))
}
# define some innner-paras

In [6]:
def myrnn(x,weights,biases):
    #:(batch_size,n_steps,n_inputs)
    x=tf.reshape(x,[-1,n_inputs])
    xin=tf.matmul(x,weights['in'])+biases['in']
    xin=tf.reshape(xin,shape=[-1,n_steps,n_hidden_units])
    # in above
    lstmcell=tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_units,forget_bias=1,
                                          state_is_tuple=True)
    #state si tuple yuanzu
    #nsamplevalue=tf.unpack(nsample)
    #_init=lstmcell.zero_state(nsamplevalue,dtype=tf.float32)
    out,states=tf.nn.dynamic_rnn(cell=lstmcell,inputs=xin,initial_state=init_state,
                                 time_major=False)
    out=tf.unpack(tf.transpose(out,[1,0,2]))
    re=tf.nn.softmax(tf.matmul(out[-1],weights['out'])+biases['out'])
    return re
    
    

In [7]:
prediction=myrnn(x,weights,biases)
loss=tf.reduce_mean(-tf.reduce_sum(y*tf.log(prediction),reduction_indices=[1]))
train=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

inivar=tf.initialize_all_variables()
sess=tf.Session()
sess.run(inivar)
step=0;
while step<train_iters:
    batch_xs,batch_ys=mnist.train.next_batch(batch_size)
    batch_xs=batch_xs.reshape([batch_size,n_steps,n_inputs])
    sess.run(train,feed_dict={x:batch_xs,y:batch_ys,init_state: np.zeros([batch_size, 2*n_hidden_units])})
    vxs=mnist.test.images
    vxs=vxs.reshape([-1,n_steps,n_inputs])
    
    vys=mnist.test.labels
    if step%dis_step==0:
        print(sess.run(loss,feed_dict={x:vxs,y:vys,init_state: np.zeros([vxs.shape[0], 2*n_hidden_units])}))
    step+=1
        

TypeError: 'Tensor' object is not iterable.